## 1) Import modules

In [ ]:
using SudokuPlantDesign
using BenchmarkTools
using DataFrames
using Random
using XLSX
using PyCall
using ColorTypes
@pyimport matplotlib.colors as matcolors

## 2) Define Trial sudoku data and define configuration

In [ ]:
#define trialname
trialname="Testtrial_2021"
#define number of genotypes which are completely replicated
numberofchecktypes=15
#define name of list with entries and checks
templatename="Sample_data_Sudoku-RCBD.xlsx"
#define color map
cmap_sudoku=ColorMap("Sudoku",[RGB(0.5,0.5,0.5),RGB(1, 0.933, 0.058),
        RGB(0.956, 0.164, 0.866),
        RGB(0.113, 0.666, 0.239),
        RGB(0.560, 0.368, 0.831),
        RGB(0.058, 0.733, 1),
        RGB(1,0,0),])
# define trial layout (x direction, y direction)
# we create here a design with 45 plots in x-direction and 3 plots in y-direction.
# one block contains 15 plots and goes along 3 y-coordinate points and 5 x-coordinate points.
C = get_configuration([5,5,5], [3], numberofchecktypes)
#display random distribution of checks within blocks (=classical augmented design)
initialize_checks_per_block!(C)
#specify empty fields
#empty_plots!(C, 11:20,1:6)
show_configuration(C, zoom=0.8,cmap=cmap_sudoku)
savefig("initial_design_random_"*trialname*".pdf")

## 3) Define sudoku functions

In [ ]:
function mc_update!(C, T, updatemoves, K)
    K_alt = K(C)
    move  = rand(updatemoves)
    new_move!(move, C)
    apply_update!(C, move)
    K_neu = K(C)

    alpha = exp(-(K_neu - K_alt)/T)
    if rand() < alpha
        # akzeptieren
    else
        # nicht akzeptieren
        reverse_update!(C, move)
    end
    return nothing
end

beta(t) = 1*exp(-7*t) * (1 + 0.2*cos(t*30*pi))
tvals = 0:0.001:1
plot(tvals, beta.(tvals))
xlabel("rel. update")
ylabel("T")

## 4) Specialize update moves and costs

updates = [
    #UpdateMoveNewCheckLabel(),
    #UpdateMoveInsertCheck(),
    #UpdateMoveRemoveCheck(),
    UpdateMoveSwapCheckCheck(),
    #UpdateMoveSwapCheckGenotype()
]

function K_indiv(conf :: C) :: Float64 where {C <: CheckConfiguration}
    return  K_num_checks_equal_per_type(conf) +
            K_checks_per_type_per_block(conf, 1)*20+
            #K_neighbors_different_check_dmax_const(conf, 1, 0.5) +
            K_neighbors_different_check_functional(conf, d->0.5/(d^3)) +
            K_neighbors_same_check_functional(conf, d->1/(d^3))*12
end

## 4) Run sudoku

In [ ]:
kosten = zeros(100000)

#specify how many checks: (x*y)-no.genotypes - missing plots

#initialize_checks!(C, numberofchecks)

#number of genotypes
#initialize_genotypes!(C, numberofgenotypes)
@showprogress "Optimizing Design ... " for i in 1:length(kosten)
    mc_update!(C, beta(i/length(kosten))*0.5, updates, K_indiv)
    kosten[i] = K_indiv(C)
end
kosten .-= minimum(kosten)
kosten .+= 1

#print things
println("Kosten: ",K_indiv(C))
println("perc_checks: ",C.num_checks_total / (C.num_plots_total - C.num_checks_total))
println("genotypes: ",C.num_plots_total - C.num_checks_total)
println("checks:    ",C.num_checks_total, "  (", round(100*C.num_checks_total / C.num_plots_total, digits=2), "%)")
for i in 1:C.N
    println("check ($(i)): ",C.num_checks[i], "  (", round(100*C.num_checks[i] / C.num_checks_total, digits=2), "%)")    
end

show_configuration(C,cmap=cmap_sudoku,zoom=0.8)


figure()
plot(kosten)
yscale("log")


#save without check labels
show_configuration(C,check_labels=false,cmap=cmap_sudoku)
mkpath("./output")
savefig("output/final_sudoku_design_"*trialname*".pdf")


## 5) Save field plan and design data

In [ ]:

#optional: load existing configuration
#load_configuration(C, "configuration_"*trialname*".txt")

#read in check data
checkdata = DataFrame(XLSX.readtable(templatename, "checks")...)
checknames=names(checkdata)
for i in 1:length(checknames)
    checkdata[!,Symbol(checknames[i])]=string.(collect(checkdata[!,Symbol(checknames[i])]))
end


#replace missing with NA
for col in eachcol(checkdata)
    replace!(col,missing => "NA")
end


if nrow(checkdata)!= numberofchecktypes
    println("Number of checks in csv not matching pre-defined checks!")
end


#get the Plant label
#POS = get_POS_block_x(C,posempty=true) #if over 2 big blocks in splitted by x
POS = get_POS_coordinatewise(C,"x","left","lower","snake") #with robot
L = get_Plant_Code_Label_allinfo_checksonly(C,POS,checkdata)

show_pos_and_labels_allinfo(C, POS, L, zoom=0.9,fontsize=7,alpha=0.5,cmap=cmap_sudoku)
savefig("./output/final_sudoku_design_labels_"*trialname*".pdf")

In [ ]:
#write file to xlsx
write_to_xlsx_allinfo(C.configuration,trialname,POS,L)

#save configuration
save_configuration(C, "./output/configuration_"*trialname*".txt")
